In [36]:
import websockets
import asyncio
import requests
import os
from dotenv import load_dotenv
from datetime import datetime
from pydub import AudioSegment
from pydub.utils import which
from pydub.playback import play
import io
import json

AudioSegment.converter = which("ffmpeg")  # Set ffmpeg path
AudioSegment.ffprobe = which("ffprobe") 
load_dotenv()

True

In [77]:


# send auth post request 

PLAYHT_API_KEY = os.getenv("PLAYHT_API_KEY")
PLAYHT_USER_ID = os.getenv("PLAYHT_USER_ID")

def auth():
    url = "https://api.play.ht/api/v4/websocket-auth"
    headers = {
        "Authorization": "Bearer " + PLAYHT_API_KEY,
        "X-User-Id"    : PLAYHT_USER_ID,
        "Content-Type" : "application/json"
        }
    
    response = requests.post(url, headers=headers)

    return response.json()


response = auth()

In [75]:

voice_dict = {
    "GordonRamsay": "s3://voice-cloning-zero-shot/1806c2a9-2c23-4336-93a5-5447b80295e9/original/manifest.json",
    "DavidGoggins": "s3://voice-cloning-zero-shot/f18bfd62-800f-4c0c-a735-ed379699a0be/davidgoggins/manifest.json",
}

async def handle_request(text, model, path, voice="DavidGoggins"):
    # takes input text, passes it to tts model, receives audio file over websocket


    # if time diff is less than an hour, renew token

    url = response['websocket_urls'][model]



    request = {
         "text": text, # less than 20k characters
         "voice": voice_dict[voice], #GordonRamsay clone
         "output_format": "mp3",
         "temperature": "0.7",
         "speed": "1.0",
    }

    audio_chunks = []

    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps(request))
        while True:
            message = await websocket.recv()

            if isinstance(message, bytes):
                    # Received audio chunk
                    audio_chunks.append(message)
            else:
                # Received a JSON message
                data = json.loads(message)

                if data.get("type") == "start":
                    print(f"Processing started. Request ID: {data.get('request_id')}")
                    print(data)

                elif data.get("type") == "end":
                    print(f"Processing ended. Request ID: {data.get('request_id')}")
                    break

        # Combine received audio chunks
        audio_data = b"".join(audio_chunks)
        

        # Convert binary data into an audio segment
        audio_segment = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")

        # Save or play the audio
        audio_segment.export(path, format="mp3")

        # Play the audio
        # play(audio_segment)

In [43]:
response['websocket_urls'].keys()
model = "PlayDialog"

In [48]:
text = """You're such a fucking twat, you should've been aborted."""

await handle_request(text, model)
    

Processing started. Request ID: e2971473-1403-427b-a873-48a422e6dd5f
Processing ended. Request ID: e2971473-1403-427b-a873-48a422e6dd5f
Raw audio saved as output_raw.mp3
Audio saved as output.mpeg


In [ ]:
import requests

headers = {
        "Authorization": "Bearer " + PLAYHT_API_KEY,
        "X-User-Id"    : PLAYHT_USER_ID,
        "Content-Type" : "application/json"
        }

url = "https://api.play.ht/api/v2/cloned-voices"



response1 = requests.get(url, headers=headers)

print(response1.text)

[{"id":"s3://voice-cloning-zero-shot/1806c2a9-2c23-4336-93a5-5447b80295e9/original/manifest.json","name":"GordonRamsay","type":"instant","gender":"male","voice_engine":"PlayHT2.0"}]


In [52]:
from groq import Groq

groq_client = Groq(
    api_key=os.getenv("GROQ_API")
)

personality = "David Goggins"

RESPONSES = []

for i in range(1):
    text = "You are {personality}: a strict, no-nonsense motivator with snark but zero offensive language. Don't say anything. Just give me quotes David Goggins would say if he found someone slacking off on his phone or on social media"
    new_message = {

        "role": "user",
        "content": f"{text} Answer in less than 20, give me 20 responses. Again answer less than 20 words. Give them to me in a python list, withou"
        
    }


    chat_completion = groq_client.chat.completions.create(
        messages = [new_message],
        model="deepseek-r1-distill-llama-70b"
    )

    response = chat_completion.choices[0].message.content
    deeseek_think = response.split("</think>")[0]
    response = response.split("</think>")[1].lstrip().rstrip()
    RESPONSES.append(response)



In [54]:
responses = RESPONSES[0].split(".")

# for i in range(len(responses)):
#     responses[i] = responses[i].lstrip().rstrip()[1:-4]

In [56]:
print(RESPONSES[0])

```python
[
    "You're not grinding, you're just spinning.",
    "Focus or fail, your choice.",
    "Stop scrolling, start climbing.",
    " Discipline equals freedom, remember?",
    "No excuses, only results.",
    "Hard work beats talent every time.",
    "You're wasting time, not improving.",
    "Comfort kills champions, stay hungry.",
    "Your phone won't get you to the top.",
    "Greatness isn't found in likes.",
    "Stop distracting yourself, start dominating.",
    "The grind doesn't stop, neither should you.",
    "No one cares about your excuses.",
    "Success is earned, not downloaded.",
    "You're either all in or all out.",
    "Your future is built, not browsed.",
    "Stay focused, stay fierce, stay disciplined.",
    "Stop procrastinating, start producing.",
    "Hard work is the only shortcut.",
    "Your potential is wasted on screens."
]
```


In [84]:
responses = [
    "You're not grinding, you're just spinning.",
    "Focus or fail, your choice.",
    "Stop scrolling, start climbing.",
    " Discipline equals freedom, remember?",
    "No excuses, only results.",
    "Hard work beats talent every time.",
    "You're wasting time, not improving.",
    "Comfort kills champions, stay hungry.",
    "Your phone won't get you to the top.",
    "Greatness isn't found in likes.",
    "Stop distracting yourself, start dominating.",
    "The grind doesn't stop, neither should you.",
    "No one cares about your excuses.",
    "Success is earned, not downloaded.",
    "You're either all in or all out.",
    "Your future is built, not browsed.",
    "Stay focused, stay fierce, stay disciplined.",
    "Stop procrastinating, start producing.",
    "Hard work is the only shortcut.",
    "Your potential is wasted on screens."
]

for i in range(16, len(responses)):
    await handle_request(responses[i], model, "./davidGoggins/{}.mp3".format(i+1))


Processing started. Request ID: 5e7b02e7-f68f-4509-9975-ee59a81149f5
{'type': 'start', 'request_id': '5e7b02e7-f68f-4509-9975-ee59a81149f5', 'status': 200, 'headers': {'Date': 'Sun, 09 Feb 2025 10:10:07 GMT', 'Server': 'uvicorn', 'x-play-job-id': 'd9f45dfe-9f07-4b56-8d85-70ca8f717e1b', 'Content-Type': 'audio/mpeg', 'x-fal-runner-hints': '/data/.playht_cache/s3/play-fal/ldm-models/LDMGuidedFT_ReGFT3_HFAB160k_VCMT43k_47000_1.9673_no_optim.gguf-None,/data/.playht_cache/s3/play-fal/ldm-models/v075_g_01560000', 'Transfer-Encoding': 'chunked', 'X-Fal-Served-From': '3558fb42-1b1b-40da-ade5-eb3719080d4f', 'x-fal-request-id': '5e7b02e7-f68f-4509-9975-ee59a81149f5'}}
Processing ended. Request ID: 5e7b02e7-f68f-4509-9975-ee59a81149f5
Processing started. Request ID: 06e5e044-06bb-4710-8d94-f5a25525a293
{'type': 'start', 'request_id': '06e5e044-06bb-4710-8d94-f5a25525a293', 'status': 200, 'headers': {'Date': 'Sun, 09 Feb 2025 10:10:10 GMT', 'Server': 'uvicorn', 'x-play-job-id': 'e4b0c8ed-9eee-4545-

In [67]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API credentials
PLAYHT_API_KEY = os.getenv("PLAYHT_API_KEY")
PLAYHT_USER_ID = os.getenv("PLAYHT_USER_ID")

# Ensure credentials are loaded
if not PLAYHT_API_KEY or not PLAYHT_USER_ID:
    raise ValueError("Missing API credentials. Check your .env file.")

# API Endpoint
api_url = "https://api.play.ht/api/v2/cloned-voices/instant/"

# Headers
headers = {
    "Authorization": "Bearer " + PLAYHT_API_KEY,
    "X-User-Id": PLAYHT_USER_ID
}

# Replace with the actual direct download link from GoFile
sample_file_url = "https://srv-file11.gofile.io/download/XFO5ZB/"

# Request Payload
data = {
    "sample_file_url": sample_file_url,
    "voice_name": "DavidGoggins"
}

# Send API request (use json= instead of data=)
response = requests.post(api_url, headers=headers, json=data)

# Print API Response
print(response.json())


{'error_message': "Sample file must be one of the following audio formats: 'audio/aac', 'audio/mpeg', 'audio/ogg', 'audio/wav', 'audio/webm', 'audio/flac', 'audio/midi', 'audio/x-midi', 'audio/mp4', 'audio/x-m4a', 'audio/x-ms-wma', 'audio/vnd.wave', 'audio/amr', 'audio/x-aiff', 'audio/x-wav', 'audio/x-pn-wav', 'audio/x-pn-realaudio', 'audio/x-mpegurl', 'video/mov'. Please refer to https://docs.play.ht/reference/api-create-instant-voice-clone for more info.", 'error_id': 'INVALID_PARAMETERS'}


In [74]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API credentials
PLAYHT_API_KEY = os.getenv("PLAYHT_API_KEY")
PLAYHT_USER_ID = os.getenv("PLAYHT_USER_ID")

# Ensure credentials are loaded
if not PLAYHT_API_KEY or not PLAYHT_USER_ID:
    raise ValueError("Missing API credentials. Check your .env file.")

# API Endpoint
api_url = "https://api.play.ht/api/v2/cloned-voices/instant/"

# Headers
headers = {
    "Authorization": "Bearer " + PLAYHT_API_KEY,
    "X-User-Id": PLAYHT_USER_ID,
    "content-type": "multipart/form-data",
    "sample_file_url": sample_file_url,
    "voice_name": "DavidGoggins",
}

# Replace with the actual direct download link from GoFile
sample_file_url = "https://drive.google.com/uc?export=download&id=1b6R7tU1J9PSf1DJMN3IvpUXx_G-J6JT1"

# Request Payload
data = {
    "sample_file_url": sample_file_url,
    "voice_name": "DavidGoggins"
}

# Send API request (use json= instead of data=)
response = requests.post(api_url, headers=headers, json=data)

# Print API Response
print(response.json())


{'error_message': 'Missing `voice_name`. Please refer to https://docs.play.ht/reference/api-create-instant-voice-clone for more info.', 'error_id': 'INVALID_PARAMETERS'}


In [ ]:
curl --request POST \
     --url https://api.play.ht/api/v2/cloned-voices/instant \
     --header 'accept: application/json' \
     --header 'content-type: multipart/form-data'